# Accuracy Benchmarking Pre-Compiled Models

Here, we provide a step-by-step tutorial on how to run pre-compiled models using any of the supported runtime engines to obtain accuracy benchmarks. 

## Select the Task Type

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import ipywidgets as widgets
from jai_benchmark import *
import configs

tasks_list = ['classification', 'detection', 'segmentation']

selected_task_id = widgets.Dropdown(
    options=tasks_list,
    value=tasks_list[0],
    description='Select Task Type:',
    disabled=False,
)
display(selected_task_id)


## Select the RunTime Engine

In [ ]:
rt_list = ['tflitert', 'tvmdlr', 'onnxrt']

selected_rt_id = widgets.Dropdown(
    options=rt_list,
    value=rt_list[0],
    description='Select Runtime Type:',
    disabled=False,
)
display(selected_rt_id)

## Select the Dataset to use

In [ ]:
valid_dataset_list = datasets.get_dataset_categories(settings=None, task_type=selected_task_id.value)

selected_dataset_id = widgets.Dropdown(
    options=valid_dataset_list,
    value=valid_dataset_list[0],
    description='Select Dataset Type:',
    disabled=False,
)
display(selected_dataset_id)

## Download Dataset and Setup Pipeline

When this cell is executed, if the dataset is already available, it is not downloaded. Download and data preparation will take approximately 45 - 60 minutes.

<div class="alert alert-block alert-info">
<b>Note:</b> You cannot interrupt the download once it is in progress.
</div>

In [ ]:
num_frames = 100

settings = config_settings.ConfigSettings('./configs/accuracy_infer_on_j7.yaml', 
                                          num_frames=num_frames, 
                                          task_selection = selected_task_id.value,
                                          dataset_loading = selected_dataset_id.value)
_ = datasets.download_datasets(settings)

## Select Model

In [ ]:
work_dir = os.path.join('prebuilt-models', f'{settings.tidl_tensor_bits}bits')

prebuilt_configs = configs.select_configs(settings, work_dir, selected_rt_id.value)
merged_list = utils.get_name_key_pair_list(prebuilt_configs.keys(), selected_rt_id.value)

selected_model_id = widgets.Dropdown(
    options=merged_list,
    value=merged_list[0][1],
    description='Select Model:',
    disabled=False,
)
display(selected_model_id)

## Perform inference to get Accuracy Benchmarks
Once the selections are made, execute the code segment below to get accuracy benchmarks. Below are the accuracy statistics that will be displayed based on the selected task:
- Image Classification - Top 1% accuracy
- Object detection - mAP % at Various Box overlaps \[0.5 to 0.95\]
- Semantic Segmentation - mIOU % (Mean IOU)

In [ ]:
pipeline_configs = {selected_model_id.value:prebuilt_configs[selected_model_id.value]}
pipeline_runner = pipelines.PipelineRunner(settings, pipeline_configs)

_ = pipeline_runner.run()